In [3]:
from typing import Callable, Generic, TypeVar

# 定義回呼函式的輸入與輸出類型
T = TypeVar("T")  # 回呼函式的輸入類型
R = TypeVar("R")  # 回呼函式的回傳值類型


class EventHandler(Generic[T, R]):
    """事件處理器"""

    def __init__(self) -> None:
        # 存儲事件與其對應的回呼函式
        self._callbacks: list[Callable[[T], R]] = []

        # 使用 TypeVar 的 __bound__ 屬性來儲存類型資訊
        self._input_type = T.__bound__ if hasattr(T, "__bound__") else object
        self._output_type = R.__bound__ if hasattr(R, "__bound__") else object

    def register(self, cb: Callable[[T], R]) -> None:
        """註冊回調函數

        Args:
            cb (Callable[[T], R]): 回調函數
        """
        self._callbacks.append(cb)

    def unregister(self, cb: Callable[[T], R]) -> None:
        """取消註冊回調函數

        Args:
            cb (Callable[[T], R]): 回調函數
        """
        self._callbacks.remove(cb)

    def unregister_all(self) -> None:
        """取消所有回調函數"""
        self._callbacks.clear()

    def trigger(self, *args: T, **kwargs: T) -> None:
        """觸發所有回調函數

        Args:
            *args(T): 可變數量的位置參數，例如 trigger(1, 2, 3) 中的 1, 2, 3
            **kwargs(T): 可變數量的關鍵字參數，例如 trigger(x=1, y=2) 中的 x=1, y=2
        """
        for cb in self._callbacks:
            cb(*args, **kwargs)


In [6]:
from typing import Any


def test_handler1(*args, **kwargs):
    print("處理器1:", args, kwargs)


# 創建事件處理器實例
event_handler = EventHandler[Any, Any]()
print(f"input_type: {event_handler._input_type}")
print(f"output_type: {event_handler._output_type}")

# 註冊處理器
event_handler.register(test_handler1)

# 測試觸發
event_handler.trigger(1, x=2)  # 觸發所有處理器

input_type: None
output_type: None
處理器1: (1,) {'x': 2}


In [7]:
def test_handler3(data: tuple[int, float]) -> None:
    print("處理器3:", data)


def test_handler4(data: tuple[int, float]) -> None:
    print("處理器4:", data)


# 創建新的事件處理器實例
event_handler2 = EventHandler[tuple[int, float], None]()

# 註冊處理器
event_handler2.register(test_handler3)
event_handler2.register(test_handler4)

# 測試觸發
test_data1 = (1, 2.0)
test_data2 = (3, 4.0)
test_data3 = (5, 6.0)

event_handler2.trigger(test_data1)  # 觸發所有處理器
event_handler2.trigger(test_data2)  # 觸發特定處理器
event_handler2.trigger(test_data3)  # 觸發特定處理器

處理器3: (1, 2.0)
處理器4: (1, 2.0)
處理器3: (3, 4.0)
處理器4: (3, 4.0)
處理器3: (5, 6.0)
處理器4: (5, 6.0)
